# Scheduling execution of tasks with QCG-PilotJob

In this Notebook we present how to use the QCG-PilotJob to schedule and execute set of tasks on pre-allocated resources. Typically, those resources are from an allocation created in a scheduling system, but can be also (mostly for the tests) defined on top of resources available locally.

## Installing QCG-PilotJob

Before we start our examples, we need to install QCG-PilotJob. The QCG-PilotJob modules can be easily installed with pip:

In [ ]:
pip install --upgrade qcg-pilotjob

# Batch mode

QCG-PilotJob allows to schedule and execute jobs in a batch mode. In this mode we prepare the description of jobs we want to run, along with their resource requirements and relationships as an input file in JSON format and run the QCG-PilotJob Manager service (`qcg-pm-service` command) with this file as an argument. 

## Execution Scenario

In the following example we want to run a workflow which consists of a few tasks, including separate tasks for computations and visualisation of results. In the first step we take the colloid example for *Lammps* package, where we simulate collisions of big particles with the small ones. This simulation produces a bunch of images presenting the simulation progress. The second step is to prepare the images for the rendering a movie - in order to make the next steps strightforward we rename the files by removing the trailing zeros from the names. In the following step we will render a movie based on the images - we will use the *ffmpeg* application for this. The last step will be removing the images produced by the simulation.

The sample file containing description of our scenario we put in the *lammps.json* file. Below we present the contents of this file.

In [ ]:
!cat examples/lammps.json

The **jobs** list in the **submit** request contains 4 elements, each representing a job we want to run. As we can see each job has a similar structure and it consists of the following elements:
* **name** of a job
* execution parameters (**execution** element) which contains name of the program to execute (**exec** element, with program arguments - the **args** list) or commands to execute (**script** element), the files where to/from redirect **stdin**, **stdout** and **stderr** streams and working directory (**wd** element) where to run job - if working directory contains relative path it will always be relative to the working directory of QCG PilotJob Manager
* dependencies parameters (**dependencies** element) where **after** list contain list of jobs which must finish (correctly, with exit code 0) before the job start

The second, **control** request, **finishAfterAllTasksDone** tells QCG-PilotJob Manager to stop listening for new requests and terminate after all submitted jobs finish.

## Running the service in the batch mode

Firstly, to make sure the working directory from previous runs is not present we make an attempt to delete it

In [ ]:
!rm -fR lammps-example-out

The only required parameter for running QCG-PilotJob Manager in the batch mode is **--file-path** which should point to the file with jobs description. In our example we will use the following two optional parameters for convencience:
* **--wd** - points the working directory where output and temporary files should be written, all relative jobs' working directories will be created as subdirectories in the service's working directory
* **--show-progress** - print the progress of executing jobs

The resulting command, executing the whole workflow, will look as the one shown below. Its execution should took less than minute. 

In [ ]:
!qcg-pm-service --wd lammps-example-out --show-progress --file-path examples/lammps.json

Now the output movie should be generated as **colloid.mp4** file in the `lammps-example-out/lammps` directory. We can display it with the following command:

In [ ]:
from IPython.display import Video
Video("lammps-example-out/lammps/colloid.mp4")

# API mode

QCG-PilotJob offers programming interfaces which allow to communicate with the service during runtime from a client program. 
We will focus on a description of the main API which covers all functionalities offered by QCG-PilotJob. 
In the latter part of the section we will demonstrate also an alternative, Executor-like API, which is based on the main API, but may be more conventient for simpler use-cases. 

## Instantiation of QCG-PilotJob Manager

There are two options in regards to instantation of QCG-PilotJob Manager service:
* QCG-PilotJob Manager can be created in advance (for example from commandline) and from the code we just connect to this already running instance. 
* QCG-PilotJob Manager can be created at the beginning of the program. 

The second approach is more convinient, so we will use it in the following examples. 

The first step in ofder to use the API is creation of the **LocalManager** object. It will launch the QCG-PilotJob Manager service in a background thread and expose a set of methods allowing us to communicate with it. 

In [ ]:
from qcg.pilotjob.api.manager import LocalManager

m = LocalManager()

## Avilable resources

By default QCG-PilotJob uses all resources available to the service. Thus, depending on the place of its execution, it will use all resources available in the scheduling system's allocation or all resources available on a local computer.

In order to get information about available resources, you can query the manager service using the **resources** method:

In [ ]:
print(m.resources())

## Submitting jobs
To submit a jobs, we need to create a **Jobs** object with jobs descriptions we want to submit:

In [ ]:
from qcg.pilotjob.api.job import Jobs
jobs = Jobs()

We can define elementary jobs descriptions, either as keyword arguments:

In [ ]:
jobs.add(name='first', exec='date', stdout='first.out')

or by passing the dictionary with arguments:

In [ ]:
jobs.add({'name': 'second', 'script': 'echo "the second job"', 'stdout': 'second.out'})

Now, with the **submit** method we can pass a group of jobs for the execution:

In [ ]:
ids = m.submit(jobs)

In return we get the identifiers of submitted jobs. Whenever we used  the **name** parameter for our job description, the associated identifier for the submitted job will have the same value. If the **name** was not used, the identifier will be defined automatically. 

In [ ]:
print(ids)

## Getting information about jobs
We can ask QCG-PilotJob Manager about statuses of jobs with the **status** method, for which we pass the identifiers of the jobs:

In [ ]:
status = m.status(ids)
print(status)

It is also possible to get more detailed information about the jobs, with the **info** method:

In [ ]:
info = m.info(ids)
print(info)

In return we will got a dictionary containing the current status of our jobs along with the additional data such as:
* information about the runtime of each job (**runtime**/**rtime**)
* information about allocated resources for a job (**runtime**/**allocation**)
* the history of a job status changes (**history**)

To display this data in a more user-friendly form, we can invoke the **info_parsed** method, which will decode runtime data for each job into a collection of **JobInfo** elements:

In [ ]:
pinfo = m.info_parsed(ids)
for job_name, job_info in pinfo.items():
    print('job name: {}'.format(job_name))
    print('total cores: {}'.format(job_info.total_cores))
    print('run time: {}'.format(job_info.time))

## Waiting for jobs completion

The **submit** method is non-blocking. In order to know if submitted jobs are compltted you can query for their status, but you can also use more convenient way that are **wait4** and **wait4all** methods. The former one will wait for the completion of specified jobs, while the latter for all jobs that are submitted to QCG-PilotJob Manager. Let's wait a while for a sample `sleep` job:

In [ ]:
ids = m.submit(Jobs().add(name='sleep_job', exec='/bin/sleep', args=['10']))
m.wait4(ids)

## Iterative jobs

QCG-PilotJob Manager allows to run a bunch of jobs which have the same execution program and resource requirements in a one request. To use this feature we need to specify the **iteration** element in a job description with either the number of iterations or a range of numbers representing the iterations start point and end point respectively. In the following example we will run 10 instances of a program.

In [ ]:
ids = m.submit(Jobs().add(name='iter_job', exec='/bin/echo', stdout='echo.out.$it', args=['$it'], iteration=10))
m.wait4(ids)

One can note the usage of **$it** variable in the job description. This variable is an interation identifier placeholder which will be replaced by a concrete iteration identifier in a resulting job description. In this example we have used this identifier both to generate a name postfix for the output files and to define the parametrised input argument.  
As we can see, the set of  output files `echo.out.*` has been created and each of them should contain the proper iteration number in a postfix:

In [ ]:
!find . -name "echo.out.*"

In order to get basic information about the iteration job you can use the **info** method in the same way as for the regular jobs:

In [ ]:
print(m.info(ids))

In case of iteration jobs, the output of **info** contains also information about iteration jobs: the range of iterations, the total number of iterations, the numbers of finished and failed jobs. 

If we need detailed information about individual iterations we can request such information using the **info_parsed** method with **withChilds** parameter, for example:

In [ ]:
pinfo = m.info_parsed(ids, withChilds=True)
for job_name, job_info in pinfo.items():
    for child in job_info.childs:
        print(child)

## Parallel jobs

Until now our jobs used only a single core to execute. It is however easily possible to define jobs that need many nodes and cores to execute. The number of nodes and cores can be specified as exact number or as a range, where the QCG-PilotJob Manager service will try to allocate as much resources (in the specified range) as currently are available. To ilustrate a parallel execution we will use a simple MPI application that will print information about a number of instances launched. The code is available in the `examples/mpi/mpi_print.c` file.

In [ ]:
!cat examples/mpi/mpi_print.c

Let's compile the program:

In [ ]:
!mpicc -Wall -o examples/mpi/mpi_print examples/mpi/mpi_print.c

Now we can submit a job where we run this code with *mpirun*. At the beginning we will run it serially on a single core. To do this, we provide **-n 1** arguments for *mpirun*. 

In [ ]:
mpi_single_ids = m.submit(Jobs().add(name='mpi_single', exec='mpirun', args=['-n', '1', 'examples/mpi/mpi_print'], stdout='mpi_single.out'))
m.wait4(mpi_single_ids)
mpi_single_info = m.info_parsed(mpi_single_ids)['mpi_single']
print('mpi_single total cores: {}'.format(mpi_single_info.total_cores))

and check the output file:

In [ ]:
!cat mpi_single.out

Let's move to the parallel execution. For example we may want to run our application on two cores. To this end we need to extend our job's description in two places: 
* we need to pass the **numCores=2** keyword argument for the job description that will instruct QCG-PilotJob Manager to allocate two cores for our job;
* we need to pass **-n ${ncores}** as the argument for *mpirun* - the value of this argument will be substituted by a concrete value established by QCG-PilotJob Manager, in our case it will be just **2**.

In [ ]:
mpi_parallel_ids = m.submit(Jobs().add(name='mpi_parallel', exec='mpirun', args=['-n', '${ncores}', 'examples/mpi/mpi_print'], \
                                       stdout='mpi_parallel.out', numCores=2))
m.wait4(mpi_parallel_ids)
mpi_parallel_info = m.info_parsed(mpi_parallel_ids)['mpi_parallel']
print('mpi_parallel total cores: {}'.format(mpi_parallel_info.total_cores))

and check the output file:

In [ ]:
!cat mpi_parallel.out

> It's worth to mention that mpirun (in its standard configuration, outside of scheduling system) without **-n** parameter will run as many instances of the program as all available cores in the system.

In order to simplify execution of MPI apps on HPC machines, QCG-PilotJob provides also a set of specialised execution models for different MPI flavours. When a specialised model is selected inside a queuing system allocation, it releases a user from the specification of resources numbers and possibly other specific parameters. Whenever possible (when `srun` can be used to start MPI applications) it is recommended to select `srunmpi` model.

Consequently the definition of a job from previous example could be modified to the simplified form, as presented below. **Please note that this example is just for reference and it can't work here, as we are not in a task started by queuing system!** 

In [ ]:
mpi_single_ids = m.submit(Jobs().add(name='mpi_single', model='srunmpi', exec='examples/mpi/mpi_print', stdout='mpi_single.out'))
m.wait4(mpi_single_ids)
mpi_single_info = m.info_parsed(mpi_single_ids)['mpi_single']
print('mpi_single total cores: {}'.format(mpi_single_info.total_cores))

## Cleanup

Using QCG-PilotJob API we should take care of finishing QCG-PilotJob Manager. The API provides the **finish** method, which will send the finish message to the manager service and wait (in case of **LocalManager**) for the manager's service thread to complete.

In [ ]:
m.finish()

## Scheduling system's allocation *vs* local resources
QCG-PilogJob Manager schedules and executes jobs on available resources. By default, the information about resources is obtained automatically - the library will check the environment to look for variables set by Slurm scheduling system. In case where those variables exist, the information about Slurm allocation's resources will be inheritted and used for the QCG-PilotJob instance. In other cases, where QCG-PilotJob is executed outside the Slurm's allocation, QCG-PilotJob Manager will execute jobs on all locally available cores. 

In order to emulate execution on more resources than available in a local environment, which may be useful for testing purposes, we can pass the **-nodes** parameter for the the manager's constructor:

In [ ]:
from qcg.pilotjob.api.manager import LocalManager

m = LocalManager(['--nodes', 'node1:8,node2:8', '--log', 'error'])
print(m.resources())

The **--nodes** parameter has a format `[NODE_NAME]:CORES[,[NODE_NAME]:CORES]...`, which allows to define quite complex resources structure. 
As we can see in the example, this time we run QCG-PilotJob Manager on 2 nodes and 16 cores. We have to remember that these are actually *virtual* cores, and although QCG-PilotJob Manager will execute sixteen jobs at the same time, physically they will share the physical cores available in the environment. Consequently such an execution will be will be slower. 

In [ ]:
m.finish()

## Executor API

Executor API is an alternative programming interface for QCG-PilotJob. In some aspects it mimics an interfaces of *concurrent.futures.Executor* and *concurent.futures.Future* and may therefore be appealing to many Python programmers. However, since this interface is still under development, it is dedicated mostly for less-demanding use-cases.

Executor API is based on the basic API of QCG-PilotJob and therefore it inherits core elements from that API. On the other hand, in order to support definition of the common execution scenarios, many elements of basic API have been hidden behind a simplified interface.

### Installation

QCG-PilotJob Executor API is delivered in a separate package. It can be installed with the following command:

In [ ]:
pip install --upgrade qcg-pilotjob-executor-api

### QCGPJExecutor
**QCGPJExecutor** is an entry point to the QCG-PilotJob's Executor API. It is an approximate implementation of the *concurrent.futures.Executor* interface, but instead of execution of functions using threads or multiprocessing module like it takes place in case of python build-in executors, here we execute QCG-PilotJob’s tasks. **QCGPJExecutor** can be easily created with the following command: 

In [ ]:
from qcg.pilotjob.executor_api.qcgpj_executor import QCGPJExecutor

e = QCGPJExecutor()

In this way we have just created the executor object, which, behind the scenes, initialised QCG-PilotJob *LocalManager* with default settings. In order to enable easy configuration of the commonly changed settings, particularly those exposed by *LocalManager* several optional parameters for the QCGPJExecutor constructor are provided. One of such parameters is **resources** which maps to the **--nodes** parameter in the main API and may be useful for testing QCG-PilotJob on a local laptop.

### Submission of tasks
Having the QCGPJExecutor instance created, we are able to make use of the **submit** method and submit some tasks. Before we do this, let us briefly describe the method's conceptual basics. Frankly speaking, the method operates on two kind of arguments. The first one is a template with a set of placeholders targeted for substitution and the second one is a list of pairs consisting of a placeholder name and its value that will be used for the substitution. Let's take a look at the `BasicTemplate` class, which is delivered with the QCG-PilotJob Executor API:

```
class BasicTemplate(QCGPJTemplate):
    @staticmethod
    def template() -> Tuple[str, Dict[str, Any]]:
        template = """
        {
            'name': '${name}',
            'execution': {
                'exec': '${exec}',
                'args': ${args},
                'stdout': '${stdout}',
                'stderr': '${stderr}'
            }
        }
         """

        defaults = {
            'args': [],
            'stdout': 'stdout',
            'stderr': 'stderr'
        }

        return template, defaults
```

As you can see, the class contains a **template** function that returns **template** and **defaults**. The **template** is a JSON dictionary representing a QCG-PilotJob **submit request**. What is important, it includes a set of **${}** placeholders. These placeholders may be substituted by the parameters provided to the **submit** method, which will be described in a minute. For some of the placeholders, default values are already predefined in a **defaults** dictionary, and these parameters don’t need to be substituted if there is no concrete reason for this. The rest of placeholders, namely `{name}` and `{exec}`, don’t have default values and therefore they need to be substituted by parameters provided to **submit**.

Now, once we are familiarised with the templates idea, the submission of a simple task should be quite self-descriptive:

In [ ]:
from qcg.pilotjob.executor_api.templates.basic_template import BasicTemplate

f = e.submit(BasicTemplate.template, name='executor_sample', exec='date')

What is here interesting is the fact that the **submit** method returs a `QCGPJFuture` object - `f`, which is a reference to our submission. Using this object and the **done**, **cancel** and **result** methods, it is possible to check if the submitted task has been finished, request the cancellation of an execution or invoke blocking wait until the task is finished, respectively.

In [ ]:
f.result()

At this moment our task should be already completed, so let's check the output. According to the template, it should be in the `stdout` file. 

In [ ]:
!cat stdout

**Note:** The content of `BasicTemplate` will likely be too limited for the real use cases, for example for the parallel programs. There is however no restrictions to create a custom template and use it instead of predefined one. This task shouldn't be difficult as the tamplete may be based on the structure of `BasicTemplate`.

### Cleanup
The last step needed to release resources occupied by QCG-PilotJob Manager instance is to invoke the **shutdown** method on the executor.

In [ ]:
e.shutdown()

### Context usage of QCGPJExecutor
QCGPJExecutor implements *ContextManager’s* methods that allow for its easy usage with the **with** statements. When the **with** statement is used, python will automatically take care of releasing QCGPJExecutor’s resources. Thus the above example can be transformed to the following, very consice form:

In [ ]:
with QCGPJExecutor() as e:
    f = e.submit(BasicTemplate.template, name='executor_sample', exec='date')
    f.result()